In [6]:
!pip3 install PyQt5

     |████████████████████████████████| 43.1MB 13.3MB/s eta 0:00:01
     |████████████████████████████████| 71kB 8.2MB/s  eta 0:00:01


In [13]:
%matplotlib qt
import scipy.stats as st
import numpy as np
from scipy import fftpack
import scipy.ndimage.filters
import matplotlib.pyplot as plt
import skimage.io
import skimage.exposure
import skimage.util
import skimage.morphology
import scipy.ndimage
import cv2

In [47]:
img = np.array(skimage.io.imread("./strb1.jpg"))

se = np.array([[0,0,1,0,0],
               [0,1,1,1,0],
               [1,1,1,1,1],
               [0,1,1,1,0],
               [0,0,1,0,0]])
img = np.power(img - 1, 2)

In [48]:
fig, axes = plt.subplots(1,3)
r = np.copy(img)


img_open = skimage.morphology.binary_opening(r, se)
img_close = skimage.morphology.binary_closing(r, se)

axes[0].imshow(np.power(img-1,2), cmap='gray')
axes[1].imshow(np.power(img_open-1,2), cmap='gray')
axes[2].imshow(np.power(img_close-1,2), cmap='gray')

axes[0].set_title("original")
axes[1].set_title("open")
axes[2].set_title("close")

RuntimeError: structure and input must have same dimensionality

In [ ]:
img = np.array(skimage.io.imread("././strb1.jpg"))[:,:,0]
img = np.where(img < 155, 1, 0)
mask = np.array(skimage.io.imread("././././strb1.jpg"))[:,:,0]
mask = np.where(mask < 155, 1, 0)
mask2 = np.array([[0],
                  [0],
                  [0],
                  [0],
                  [0],
                  [0],
                  [0]])
mask3 = np.array([[1,0,0,0,1],
                  [0,1,0,1,0],
                  [0,0,1,0,0],
                  [0,1,0,1,0],
                  [1,0,0,0,1],])
structure2 = np.zeros_like(mask3)

mask4 = skimage.morphology.binary_erosion(skimage.morphology.binary_erosion(mask))

r = scipy.ndimage.binary_hit_or_miss(img, mask4,structure2)

r = np.where(r==True, 1, 0)

idx = np.argwhere(r==1)

plt.imshow(np.power(img-1,2),cmap='gray')
plt.plot(idx[:,1], idx[:,0], '+')

print("done")



In [3]:
def count_hole_coin(img_org, img_no_bg):
    r = []
    fig,ax = plt.subplots(1,1)
    for i in range(0, img_org.shape[0]):
        for j in range(0, img_org.shape[1]):
            if img_no_bg[i,j] == 0:
                tmp = j
                while img_no_bg[i,tmp] == 0:
                    tmp-=1
                new_img = skimage.morphology.flood_fill(img_org, (i,tmp), 0)
                diff = np.sum(np.bitwise_xor(new_img, img_org))
                img_org = new_img
                r.append(diff)
                skimage.morphology.flood_fill(img_no_bg, (i,j), 1,inplace=True)
                print(r)
                
                ax.imshow(img_org, cmap='gray')
                fig.canvas.draw()
                
                fig.show()
                
                fig.waitforbuttonpress()
    return r, img_org

def count_full(img):
    r = []
    fig,ax = plt.subplots(1,1)
    
    for i in range(0, img.shape[0]):
        for j in range(0, img.shape[1]):
            if img[i,j] == 1:
                new_img = skimage.morphology.flood_fill(img, (i,j), 0)
                diff = np.sum(np.bitwise_xor(new_img, img))
                if diff > 100:
                    r.append(diff)
                img = new_img
                print(r)
                
                ax.imshow(img, cmap='gray')
                fig.canvas.draw()
                
                fig.show()
                
                fig.waitforbuttonpress()
    print(r)
    return r, img

def proc_coins(full, hole):
    hist_full = np.histogram(full,bins=[0,4000,6000])
    hist_hole = np.histogram(hole,bins=[0,4000,5000,7000])
    print(hist_full)
    print(hist_hole)
    
    hole_coins = ["1 kr", '2 kr', '10 kr']
    full_coins = ["50 ore", '20 kr']
    
    values_full = [1/2,20]
    values_hole = [1,2,10]
    
    total = 0
    
    for i,val in enumerate(hist_full[0]):
        print(f"{val} coins of {full_coins[i]}")
        total += values_full[i]
    for i,val in enumerate(hist_hole[0]):
        print(f"{val} coins of {hole_coins[i]}")    
        total += values_hole[i]
    print(f"total: {total} kr")
    

In [ ]:
img = np.array(skimage.io.imread(".\\input\\money_bin.jpg"))
img = np.where(img < 155, 1, 0)

img_no_bg = skimage.morphology.flood_fill(img, (0,0), 1)
np.unique(img)


r, new_img = count_hole_coin(img, img_no_bg)
r2, new_img = count_full(new_img)




img = new_img
plt.imshow(img_no_bg, cmap='gray')

[6047]
[6047, 4349]
[6047, 4349, 6065]
[6047, 4349, 6065, 4454]
[6047, 4349, 6065, 4454, 3049]


In [19]:
proc_coins(r2,r)

(array([1, 5], dtype=int64), array([   0, 4000, 6000]))
(array([1, 2, 2], dtype=int64), array([   0, 4000, 5000, 7000]))
1 coins of 50 ore
5 coins of 20 kr
1 coins of 1 kr
2 coins of 2 kr
2 coins of 10 kr
total: 33.5 kr
